In [39]:
CATEGORY = ""
user_input = "What is STEEL PRODUCTION OF Tata Steel?"

In [40]:
from config import *

import qdrant_client as qc
import qdrant_client.http.models as qmodels
from qdrant_client.http.models import *

from azure_openai_helper import generate_answer_from_context
from model_helper import load_model
from sklearn.metrics.pairwise import cosine_similarity

In [41]:
def get_qdrant_client():
    client = qc.QdrantClient(url=URL)
    METRIC = qmodels.Distance.COSINE
    return client


client = get_qdrant_client()

In [42]:
model = load_model()
xq = model.encode(user_input,convert_to_tensor=True)
if CATEGORY == '':
    query_filter = None
else:
    query_filter=qmodels.Filter(
        must= [
            FieldCondition(
                key="Category",
                match=models.MatchValue(value=CATEGORY),
            )
        ],
    )
search_result = client.search(collection_name=COLLECTION_NAME,
                                query_vector=xq.tolist(), 
                                query_filter=query_filter,
                                limit=3,
                                with_vectors = True,
                                )
contexts =""
context_text_vector = []
context_relevance = []
for result in search_result:
    print(result)
    contexts +=  result.payload['token']+"\n---\n"
    context_text_vector.append([result.vector])
    context_relevance.append(cosine_similarity([xq.tolist()], [result.vector])[0][0])

id='18c70d89-bd44-457d-a5ac-d70689f1f04b' version=0 score=0.74053377 payload={'filename': 'Tata-Steel-Corporate-Brochure-2020-21.pdf', 'token': ' \nA flagship entity of the 150-year-old \nillustrious Tata group, Tata Steel is one of the few global steel operations that is fully integrated – from mining to manufacturing and marketing of finished products', 'type': 'pdf'} vector=[-0.060963187, -0.019345853, 0.008215866, 0.048355576, -0.01016136, -0.022955257, 0.024372628, -5.2259587e-05, -0.07638687, -0.05361891, 0.008736549, -0.03879009, 0.0091067925, 0.031653006, 0.030790254, -0.032183055, 0.0134217385, -0.06410529, -0.02667872, -0.05490372, -0.08402271, -0.049466886, -0.01338228, -0.023121273, 0.0054487363, 0.016641531, 0.013717599, 0.004491768, 0.03428683, 0.0317416, -0.07824723, 0.04660555, 0.027767539, 0.05356024, -0.07023098, 0.030789983, -0.028434426, -0.03283084, -0.05468415, -0.07355286, -0.001925659, 0.057973094, -0.02769444, 0.012824047, 0.118017614, -0.004304174, 0.013286546

In [43]:
context_relevance

[0.7405337524739829, 0.7205155421453981, 0.7058981542384062]

In [44]:
reply = generate_answer_from_context(user_input, contexts)

In [45]:
reply_vector = model.encode(reply,convert_to_tensor=True)

In [46]:
reply_vector_list = reply_vector.tolist()

In [47]:
answer_relevance = cosine_similarity([xq.tolist()], [reply_vector_list])[0][0]

In [48]:
answer_relevance

0.7896408813397561

In [49]:
groundedness = []

In [50]:
for context_vector in context_text_vector:
    groundedness.append(cosine_similarity([reply_vector_list], context_vector)[0][0])   

In [51]:
groundedness

[0.7827560156206228, 0.7776656420294148, 0.8489912000060347]

RAG Triad

In [52]:
print("###################################")
print("User Input:", user_input)
print("Groundedness",(sum(groundedness) / len(groundedness)))
print("Context Relevance", (sum(context_relevance) / len(context_relevance)))
print("Relevance", answer_relevance)

###################################
User Input: What is STEEL PRODUCTION OF Tata Steel?
Groundedness 0.8031376192186906
Context Relevance 0.722315816285929
Relevance 0.7896408813397561
